In [135]:
import pandas as pd
import os
from openpyxl import load_workbook

In [136]:
folder_path = "C:\\Users\\phamt\\Downloads\\temp\\PQ Databse Append test\\Data"

In [137]:
def fill_nulls_and_combine_headers(sheet):
    max_col = sheet.max_column
    # Initialize headers from the first row for combination
    headers_first_row = [sheet.cell(row=1, column=col).value for col in range(1, max_col + 1)]
    # Use second row values directly for the first two columns
    headers_second_row = [sheet.cell(row=2, column=col).value for col in range(1, max_col + 1)]
    headers_for_first_two = headers_second_row[:2]
    
    # Fill nulls in the first row with the previous non-null values for other columns
    for i in range(2, len(headers_first_row)):
        if headers_first_row[i] is None and headers_first_row[i-1] is not None:
            headers_first_row[i] = headers_first_row[i-1]
            
    # Combine first row (with no nulls) and second row for columns beyond the first two
    combined_headers = headers_for_first_two + [f"{headers_first_row[i]} - {headers_second_row[i]}" if headers_first_row[i] is not None else headers_second_row[i] for i in range(2, len(headers_first_row))]
    
    return combined_headers

In [138]:
def read_excel_sheets(filename, folder_path):
    file_path = os.path.join(folder_path, filename)
    wb = load_workbook(file_path, data_only=True)
    all_data = []
    for sheetname in wb.sheetnames:
        if sheetname == "Overview":
            continue  # Skip the "Overview" sheet
        sheet = wb[sheetname]
        headers = fill_nulls_and_combine_headers(sheet)
        data = pd.DataFrame(sheet.values)
        # Skip the first two header rows, apply the headers
        data = data.iloc[2:]
        data.columns = headers
        all_data.append(data)
    wb.close()
    # Combine all sheets data from the current file and add 'Filename' as the last column
    return pd.concat(all_data, ignore_index=True)

In [139]:
def process_excel_files(folder_path):
    all_files_data = []
    # Iterate through each file in the folder, skipping those that start with "~"
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx") and not filename.startswith("~"):
            all_files_data.append(read_excel_sheets(filename, folder_path))
    # Combine all data into a single DataFrame
    combined_data = pd.concat(all_files_data, ignore_index=True)
    combined_data['Filename'] = filename  # Add filename column at the end
    return combined_data

In [140]:
filename_column = combined_data.pop('Filename')
combined_data['Filename'] = filename_column

In [141]:
print(combined_data.head())
combined_data.to_excel("combined_data.xlsx", index=False)

   DOW        Date OTB - Occ %   OTB - ADR OTB - Rms OTB - Revenue  \
0  Thu  2024-02-01     0.88785  214.879158        95      20413.52   
1  Fri  2024-02-02    0.962617   247.00233       103      25441.24   
2  Sat  2024-02-03    0.859813   176.73337        92      16259.47   
3  Sun  2024-02-04    0.934579    211.5221       100      21152.21   
4  Mon  2024-02-05    0.953271  218.213333       102      22257.76   

  SDLY - Occ %  SDLY - ADR SDLY - Rms SDLY - Revenue  ...  \
0     0.831776  157.650337         89       14030.88  ...   
1     0.831776  183.573034         89        16338.0  ...   
2     0.850467   204.13022         91       18575.85  ...   
3     0.570093  144.743279         61        8829.34  ...   
4     0.728972  172.534103         78       13457.66  ...   

  Compset - Veriu Central Compset - YEHS Hotel Sydney CBD  \
0                     240                             191   
1                     260                             203   
2                     305    